In [ ]:
import numpy as np
import pandas as pd


# Helper Functions
########################################################################
########################################################################
########################################################################
def random_Transition_matrix(m):#, recursive=True
    """
    Generates a random NumPy matrix of size m x m with nonnegative elements.
    Args:
    m (int): The number of rows, columns in the matrix.
    Returns:
    numpy.ndarray: A random NumPy matrix of size m x m with nonnegative elements, where sum of element in each row is one.
    """
  # Use np.random.rand to generate random values between 0 (inclusive) and 1 (exclusive)
    # if (recursive==False):

    matrix = np.random.rand(m, m) # uniform distribution
    # want different distribution per station, distribution variable per station (eg. normal, beta)
    matrix= (matrix/matrix.sum(axis=0)).T
  # Convert the random values to integers to ensure nonnegativity
#   matrix = matrix.astype(np.float)
    return matrix

########################################################################
########################################################################

########################################################################
########################################################################
########################################################################
def Transition_time(distributions, N):

    Transition_time = np.zeros(N)

    # Fill the array with values from different distributions
    for i in range(N):
        dist_name, params = distributions[i]
        Transition_time[i] = getattr(np.random, dist_name)(**params)

    return Transition_time.astype(int)

In [ ]:
def simulate_adjusted(capacity):
  N = 1 # number of stations
  scale = 6.0
  # time unit is 1/6 min = 10 s!!!

  CarArrivalRatePerUnitTime = 65/(3*60*scale) # number of arrivals per iteration (# cars arriving per second)
  df_ParkingWaitingTime = pd.DataFrame(data=np.zeros(1), columns=["WaitingTime"])
  df_cars = pd.DataFrame(data=np.zeros(1), columns=["#NotProcessedPeopleInCar"])
  ProcessRate = 1000 # max number of people can enter to stations line in each second
  MaxLength = 1000
  ParkingCapacity = capacity

  Cars_in_ParkingLine = []
  NumberOfCarsInParkingLine = 0
  EntranceLine_list = []
  ExitLine_list = []
  EntranceLineLength_list = [0]
  OccupiedParking_list = [0]
  ParkingWaitingLineLength_list = [0]
  FullyProcessedPeople = np.zeros(2)

  # data = [[[],]*N]
  data =[[[],]*1]


  StationsNameList = [0]
  df_StationLines = pd.DataFrame(data, index=StationsNameList, columns=["CurrentLine"])
  df_StationLines.index.name = "StationNumber"

  df_StationWaitingTime = pd.DataFrame(data, index=StationsNameList, columns=["WaitingTime"])
  df_StationWaitingTime.index.name = "StationNumber"

  df_StationTime = pd.DataFrame(data, index=StationsNameList, columns=["TotalTime"])
  df_StationTime.index.name = "StationNumber"

  df_StationLineLength = pd.DataFrame(0, index=[0], columns=["LineLength"])
  df_StationLineLength.index.name = "StationNumber"

  df_StationServers = pd.DataFrame(np.array([1]).astype(int), index=[0], columns=["NumberOfServers"]) # of servers = # of people that can be served in 1 unit of time (10 seconds).
  df_StationServers.index.name="StationNumber"

  df_StationLineLength_History = df_StationLineLength.copy()
  df_StationLineLength_History.columns = ["0"]

  TotalTime = []
  TotalTime_Without_Parking = []

  Init_state =  np.zeros(N)
  Init_state = np.zeros([1]) #random_Transition_matrix(N)[0,:]
  new_state_prob = Init_state
  Transition_matrix = np.ones([1,1]) # random_Transition_matrix(N)
  distributions = [
          #('normal', {'loc': 20, 'scale': 10}),  # Normal distribution for first station
          #('normal', {'loc': 20, 'scale': 10}),  # Normal distribution for second station
          #('normal', {'loc': 20, 'scale': 10}),  # Normal distribution for third station
          #('uniform', {'low': 10, 'high': 100}),  # Uniform distribution for fourth station
          #('exponential', {'scale': 20}),  # Exponential distribution for fifth station
          #('uniform', {'low': 400, 'high': 800}),  # Uniform distribution for 1st station,
          #('uniform', {'low': 200, 'high': 400}),  # Uniform distribution for 2nd station
          #('uniform', {'low': 200, 'high': 400}),  # Uniform distribution for 3rd station
          #('uniform', {'low': 200, 'high': 400}),  # Uniform distribution for 4th station
          ('exponential', {'scale': 2*scale}),  # Exponential distribution for 1st station, nonperishable / entrance
          ('exponential', {'scale': 5*scale}),  # Exponential distribution for 2nd station, clothing
          ('exponential', {'scale': 5*scale}),  # Exponential distribution for 3rd station, meat
          ('exponential', {'scale': 0.35*scale}),  # Exponential distribution for 4th station, baked goods
          ('exponential', {'scale': 1*scale}),  # Exponential distribution for 5th station, speciality/water
          ('exponential', {'scale': 3*scale}),  # Exponential distribution for 6th station, produce
      ]



  CarID = 0
  for CurrentTime in range(int(10000)): # Studied period of time
  ########################################################################
  ########        Process How Cars Enter the Parking         #############
  ########################################################################
    if (CurrentTime<3*60*scale):#3*60 # if Active hours
      # New cars arrived and add them to the parking waiting line
      if (CurrentTime==0): CurrentCarArrivalRate = 10 # to handle heavy right taled arrival
      else: CurrentCarArrivalRate = np.random.poisson(CarArrivalRatePerUnitTime, 1)[0] # number of car arrival rate

      # create counter for new car arrived
      for NewCar in range(CurrentCarArrivalRate):
        df_ParkingWaitingTime.loc[CarID, "WaitingTime"] = 0
        Cars_in_ParkingLine.append(CarID)
        CarID += 1

      # When there are available spots, allow cars in parking waiting line to enter
      NumberOfCarsinParkingLot = sum(df_cars.loc[:,"#NotProcessedPeopleInCar"]>0) # Calculate number of cars in the parking lot, say k
      ParkingAvailbaleCapacity = int(ParkingCapacity - NumberOfCarsinParkingLot)
      NewCarsEnteringTheParking = Cars_in_ParkingLine[0:ParkingAvailbaleCapacity] # First k cars in the parking waiting line can now park their cars
      Cars_in_ParkingLine = Cars_in_ParkingLine[ParkingAvailbaleCapacity:]        # The other cars still in parking waiting line

      for WaitingCarID in Cars_in_ParkingLine:
        df_ParkingWaitingTime.loc[WaitingCarID,"WaitingTime"] += 1

      NumberOfNewCarsEnteringTheParking = len(NewCarsEnteringTheParking)
      NumberOfCarsInParkingLine = len(Cars_in_ParkingLine)

      # FullyProcessedPeople[0]=FullyProcessedPeople[0]+(CurrentCarArrivalRate-CurrentFeasibleCarArrivalRate) # it's not actually exactly fully processed,
                                                                                                # but all who left beceause they had no available choice at their last step
                                                                                                # [0] counts number of cars that left due to parking capacity constraints

      for NewCarID in NewCarsEnteringTheParking:
          NumberOfPeopleInCar=np.random.randint(1,3)
          df_cars.loc[NewCarID,"#NotProcessedPeopleInCar"] = NumberOfPeopleInCar
          for i in range(NumberOfPeopleInCar):
            NewCarPeople = np.zeros([3,N+1])
            NewCarPeople[1,-1] = NewCarID
            NewCarPeople[0,-1] = 0 # assume each person spends 4 min. at parking
            NewCarPeople[0,:-1] = Transition_time(distributions, N)
            # we randomly generate the amount of time each person spends in each station server
            # time spent at each station is btw. 30 <= t <= 240 secs

            # Skipping removed # only %25 of shoppers complete clothing (assume station 1 is for clothing)
            NewCarPeople[2,:] = NewCarPeople[0,:]
            EntranceLine_list = [NewCarPeople] + EntranceLine_list
            # entranceline_list is a list of arrays, where each array contains the data of the time a single person spends at each station
        # print(NewCarPeople)

          # EntranceLine_list=[np.zeros([N])]*CurrentArrivalRate+EntranceLine_list # modify it that some of them will leave ...
    else: EntranceLine_list = EntranceLine_list # modify it that some of them will leave ...

  ########################################################################
  ########      Process how people enter the stations' lines   ############
  ########################################################################
      # How many free spots are available in all stations, so we can atmost let that number in
    TotalFreeAvailableSpotInStations = MaxLength*N-df_StationLineLength.iloc[:, 0].sum()

    if(TotalFreeAvailableSpotInStations>0):
        ProcessNowRate = min(ProcessRate, len(EntranceLine_list), TotalFreeAvailableSpotInStations) # number of people enter to the end of stations' lines
        for _ in range(ProcessNowRate):
          person_history = EntranceLine_list[-1].copy()
          del EntranceLine_list[-1]# remove this person from list of people in the entarnce line

          Adjusted_Init_state = np.zeros(N)
          FreeStationIndex = list(np.argwhere(df_StationLineLength<MaxLength)[:,0]) # find index of free stations
          FullStationIndex = list(np.argwhere(df_StationLineLength>=MaxLength)[:,0]) # find index of full stations
          FeasibleStationIndex = list (set(FreeStationIndex).intersection(set(list(np.argwhere(person_history[0,:-1]>0)[:,0]))))
          # feasible station: A station where the person's transition matrix indicates they spend >= 0 time at that station
          InFeasibleStationIndex = list (set(FullStationIndex).union(set(list(np.argwhere(person_history[0,:-1]==0)[:,0]))))

          if(len(FeasibleStationIndex)>0):
            # EmptyStationIndex = list((df_StationLineLength < 20).values)
            Adjusted_Init_state[FeasibleStationIndex] = (0.001+Init_state[FeasibleStationIndex])/(Init_state[FeasibleStationIndex].sum()+0.001*len(FeasibleStationIndex))
            #Normalizes the probabilities by dividing each by the total sum of feasible station probabilities. The small value (0.001) is also added to the denominator to prevent division by zero.
            #Result: This line adjusts the initial state probabilities (Init_state) for stations that are feasible, ensuring that the probabilities sum to 1
            Adjusted_Init_state[InFeasibleStationIndex] = 0
            #Set the probability to zero for infeasible stations

            station = np.random.choice(StationsNameList, size=1, p=list(Adjusted_Init_state))[0]
            # randomly chooses one value from StationsNameList given the adjusted probabilities

            # person_history[0,station]+=1
            #add the person to the chosen station's line & update the length of the line
            df_StationLines.iloc[station, 0] = [person_history] + df_StationLines.iloc[station, 0]
            df_StationLineLength.iloc[station, 0]=len(df_StationLines.iloc[station, 0]) #df_StationLineLength.iloc[station, 0]+1

    for station in [0]: df_StationLineLength.iloc[station, 0]=len(df_StationLines.iloc[station, 0])
  ########################################################################
  ########      Process how they transit and exit stations' lines   ############
  ########################################################################
    new_state_prob = np.dot(new_state_prob, Transition_matrix)
    NonEmptyStationIndex = list(np.argwhere(df_StationLineLength>0)[:,0])
      #finds stations that have people in line

      # print("777777", df_StationLineLength, NonEmptyStationIndex)
      # print('***', list(df_StationLineLength.iloc[NonEmptyStationIndex].index))

    for station in NonEmptyStationIndex: #list(df_StationLineLength.iloc[NonEmptyStationIndex].index):
      PeopleInStationLineOutOfServer = (df_StationLineLength.iloc[station, 0]-min(df_StationLineLength.iloc[station, 0], df_StationServers.iloc[station, 0]))
      # number of people not being served
      for k in range(PeopleInStationLineOutOfServer):
        #updating the time a person has spent waiting in line to be served (+ 1 sec)
        df_StationLines.iloc[station, 0][k][1,station]+=1 # add how many seconds this person is staying in this station's line
      # people currently being served
      PeopleInStationServer = min(df_StationLineLength.iloc[station, 0], df_StationServers.iloc[station, 0])
      ServedPeople=0
        #initializing a counter for the # of people who have been FULLY served
      for k in range(PeopleInStationServer): # go through people who are currently being served

        # print(StaionBusyServer, k)

        # print(df_StationLines.loc[station, ["CurrentLine"]])
        #increment time the person has spent waiting to be served/being served
        #and decrement their remaining time to be served at the station
        df_StationLines.iloc[station, 0][-k+ServedPeople-1][1,station]+=1 # add to see how many seconds this person is staying in this station's line
        df_StationLines.iloc[station, 0][-k+ServedPeople-1][0,station]-=1 # deduct to see how many seconds this person left to be served by this station's server
        if(df_StationLines.iloc[station, 0][-k+ServedPeople-1][0,station]==0): #if service is complete (wait time == 0)
          #collecting their data and recording how long the person waited, then remove them from the station line
          person_history = df_StationLines.iloc[station, 0][-k+ServedPeople-1].copy()
          df_StationTime.iloc[station, 0]= [person_history[1,station]] + df_StationTime.iloc[station, 0] # records how long the person stayed at this station
          df_StationWaitingTime.iloc[station, 0]= [person_history[1,station]-person_history[2,station]] + df_StationWaitingTime.iloc[station, 0] # records how long the person waited at this station
          del df_StationLines.iloc[station, 0][-k+ServedPeople-1] # removes the person from the station's line
          #increment the count of served people
          ServedPeople +=1
          #update line length
          df_StationLineLength.iloc[station, 0]=len(df_StationLines.iloc[station, 0]) #df_StationLineLength.iloc[station, 0]-1

    # Now the first person in this line person goes to another station if she hasn't been to at least one
      # finds stations with open space that the person hasn't visited
          OpenStationIndex = list (set(list(np.argwhere(df_StationLineLength<MaxLength)[:,0])).intersection(set(list(np.argwhere(person_history[0,:-1]>0)[:,0]))))
          # print("888888   ", OpenStationIndex, station)
          if(len(OpenStationIndex)>0): # check if there are open stations
              Current_state_prob = Transition_matrix[station, :].copy() #np.zeros(N) # list(set(x) - set(y))
              # OpenStationIndex = list (set(list(np.argwhere(df_StationLineLength<MaxLength)[:,0])).intersection(set(list(np.argwhere(person_history>0)[:,0]))))
              BlockedStationIndex = list (set(list(np.argwhere(df_StationLineLength>=MaxLength)[:,0])).union(set(list(np.argwhere(person_history[0,:-1]==0)[:,0]))))
              #finds stations that are blocked (either full or previously visited)
              Current_state_prob[:]=(0.001+Current_state_prob[:])/(np.sum(Current_state_prob[:])+0.001*N)
              #normalizes the current state probabilities:
              # adds a small value to each to avoid a probability of 0 and makes all probabilities sum to 1


              Adjusted_current_state_prob=np.zeros(N)
              Adjusted_current_state_prob[OpenStationIndex] = Current_state_prob[OpenStationIndex]/Current_state_prob[OpenStationIndex].sum()
              #adjusting the state probabilities for open stations
              Adjusted_current_state_prob[BlockedStationIndex]=0
              #if blocked, the new adjusted state probability is set to 0
              new_station = np.random.choice(StationsNameList, size=1, p=list(Adjusted_current_state_prob))[0]
              #randomly selects a new stations based on the adjusted probabilities
              # print("\n **********   ",x,  new_station,(person_history), Adjusted_current_state_prob)
              df_StationLines.iloc[new_station, 0] = [person_history] + df_StationLines.iloc[new_station, 0]
              #move person to newly selected station
              df_StationLineLength.iloc[new_station, 0] = len(df_StationLines.iloc[new_station, 0]) #df_StationLineLength.iloc[new_station, 0]+1
              #update line length


          else: #if there are no more available stations, the person exits the code and is added to the exit list
            ## print(person_history)
            ExitLine_list = [person_history] + ExitLine_list
            FullyProcessedPeople[int(sum(person_history[1,:-1]>0))] = FullyProcessedPeople[int(sum(person_history[1,:-1]>0))]+1
            # it's not actually exactly fully processed, but all who left beceause they had no available choice at their last step
            # PersonCarID=person_history[1,-1]

  ########################################################################
  ########        simulate waiting time for people to get to the car         ############
  ########################################################################

    ServedPeople_=0
    CurrentExitLine_length=len(ExitLine_list)
    for kkk in range(CurrentExitLine_length):
      ExitLine_list[ServedPeople_-kkk-1][0,-1]=ExitLine_list[ServedPeople_-kkk-1][0,-1]-1
      # print(ExitLine_list[ServedPeople_-kkk-1])
      if(ExitLine_list[ServedPeople_-kkk-1][0,-1]==0):
        PersonCarID=ExitLine_list[ServedPeople_-kkk-1][1,-1]
        TotalTime.append(np.sum(ExitLine_list[ServedPeople_-kkk-1][1,:-1])+ df_ParkingWaitingTime.iloc[int(PersonCarID), 0])
        TotalTime_Without_Parking.append(np.sum(ExitLine_list[ServedPeople_-kkk-1][1,:-1]))
        # print(PersonCarID, "***", ExitLine_list[ServedPeople_-kkk-1], "\n\n")
        del ExitLine_list[ServedPeople_-kkk-1]
        ServedPeople_+=1
        df_cars.loc[PersonCarID,"#NotProcessedPeopleInCar"] = df_cars.loc[PersonCarID,"#NotProcessedPeopleInCar"]-1
        # state=np.dot(state,P)
        # print(state)
        # stateHist=np.append(stateHist,state,axis=0)
        # dfDistrHist = pd.DataFrame(stateHist)
        # dfDistrHist.plot()
        # plt.show()

    # print(df_StationLineLength)

    df_StationLineLength.columns=[str(CurrentTime+1)]
    df_StationLineLength_History=pd.concat([df_StationLineLength_History, df_StationLineLength], axis=1)
    EntranceLineLength_list=EntranceLineLength_list+[len(EntranceLine_list)]
    OccupiedParking_list=OccupiedParking_list+[sum(df_cars.loc[:,"#NotProcessedPeopleInCar"]>0)]
    ParkingWaitingLineLength_list=ParkingWaitingLineLength_list + [NumberOfCarsInParkingLine]

  AverageWaitingTime=[]
  for station in [0]:
    AverageWaitingTime+= [np.mean(df_StationWaitingTime.iloc[station,0])]
  avg_wait_all_stations = (sum(AverageWaitingTime)) / (len(AverageWaitingTime))

  AverageStationTime=[]
  for station in [0]:
    AverageStationTime+= [np.mean(df_StationTime.iloc[station,0])]
  avg_station_all_stations = (sum(AverageStationTime)) / (len(AverageStationTime))

  return avg_wait_all_stations/scale

In [ ]:
avg_wait_all_stations = simulate_adjusted(1000)


In [ ]:
avg_wait_all_stations

np.float64(14.350694444444445)

In [ ]:
def simulate_parking_adjusted():
  numsimulations = 5
  average_occupied_capacity = []
  average_waiting_time = []
  average_all_stations = []
  average_whole_pantry = []
  for capacity in range(5,31,5):
    occupied_i = []
    parkingwaiting_i = []
    all_stations_i = []
    whole_pantry_i = []
    for i in range(numsimulations):
      df_StationLineLength_History, EntranceLineLength_list, OccupiedParking_list, ParkingWaitingLineLength_list, FullyProcessedPeople, avg_wait_all_stations,avg_wait_whole_pantry, df_StationWaitingTime, df_ParkingWaitingTime, df_StationTime, TotalTime, TotalTime_Without_Parking = simulate_adjusted(capacity)
      avg_occupied = sum(OccupiedParking_list) / len(OccupiedParking_list)
      avg_parkingwaiting = df_ParkingWaitingTime.iloc[:,0].mean()
      whole_pantry_i.append(avg_wait_whole_pantry)
      occupied_i.append(avg_occupied)
      parkingwaiting_i.append(avg_parkingwaiting)
      all_stations_i.append(avg_wait_all_stations)

    average_occupied_capacity.append(sum(occupied_i) / len(occupied_i))
    average_waiting_time.append((sum(parkingwaiting_i) / len(parkingwaiting_i)) / 6) # to get from seconds to minute waiting time
    average_all_stations.append((sum(all_stations_i) / len(all_stations_i))) # to get from seconds to minute waiting time

    print(whole_pantry_i)
    average_whole_pantry.append((sum(whole_pantry_i) / len(whole_pantry_i))) # to get from seconds to minute waiting time

    print("average_occupied_capacity_parking: " + str(average_occupied_capacity))
    print("average_waiting_time_parking: " + str(average_waiting_time))
    print("average_wait_time_stations: " + str(average_all_stations))
    print("average_wait_time_pantry: " + str(average_whole_pantry))

  return average_occupied_capacity, average_waiting_time, average_all_stations, average_whole_pantry

In [ ]:
avg_occupied_capacity_parking, avg_waiting_time_parking, avg_wait_all_stations, avg_whole_pantry = simulate_parking_adjusted() # ~takes 30 minutes

[11.006877882375314, 10.88651650016029, 9.220574904952043, 13.484840791017263, 14.84404257630064]
average_occupied_capacity_parking: [9.826011102299763]
average_waiting_time_parking: [60.79390323084584]
average_wait_time_stations: [9.032631517138144]
average_wait_time_pantry: [11.88857053096111]
[13.69234544418368, 12.782038034716606, 11.733444783951567, 15.062955278479471, 13.923247238466537]
average_occupied_capacity_parking: [9.826011102299763, 14.423790642347342]
average_waiting_time_parking: [60.79390323084584, 35.00032386642841]
average_wait_time_stations: [9.032631517138144, 10.622652720617134]
average_wait_time_pantry: [11.88857053096111, 13.438806155959572]
[15.526259156726011, 16.012777087331262, 17.761566589749417, 16.423483284468237, 14.09433232203011]
average_occupied_capacity_parking: [9.826011102299763, 14.423790642347342, 18.95114988104679]
average_waiting_time_parking: [60.79390323084584, 35.00032386642841, 31.265251277923685]
average_wait_time_stations: [9.03263151713